<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/Chapter%2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10. 신경망 기계번역
* 기계번역
* seq2seq
* 어텐션
* input feeding
* 자기회귀 속성과 teacher forcing 훈련 방법
* 탐색(추론)
* 성능 평가

## 10.1. 기계번역

* 기계번역(machine translation, MT) : 자연어 처리 영역에서의 종합예술
* 규칙 기반 기계번역(RBMT)
* 통계 기반 기계번역(SMT)
* 신경망 기계번역(NMT) : 가장 큰 성취

### 10.1.1 번역의 목표
* 번역의 궁극적인 목표
    * 언어 $f$의 문장이 주어질 때, 가능한 언어 $e$로의 번역 문장 중에서 최대 확률을 갖는 $\hat{e}$ 찾아내기.
    
    $$\hat{e} = argmax P_{f\rightarrow e}(e|f) $$

* 번역이 어려운 이유
    * 인간의 언어는 컴퓨터 프로그래밍 언어처럼 명확하지 않는다.
    * 언어의 모호성 적극 활용하여 의사소통의 효율을 극대화한다.
    * 언어는 민족의 문화를 담고 있어 문화마다 차이가 발생한다.
    * 이 세가지는 기계가 우리의 말을 번역할 때 장벽이 된다.
    
* 왜 번역 기술은 중요할까?
    * 기계번역을 통해 많은 일이 일어난다.
    * 같은 세계인이 소통하는 SNS, 전 세계 대상의 인터넷 쇼핑몰 번역 서비스 제공
    * 사용자 편리함 추구

### 10.1.2 기계번역의 역사
* 규칙 기반 기계번역(RBMT)
    * 가장 전통적인 번역 방식
    * 주어진 문장의 구조 분석 $\rightarrow$ 규칙 세우고 분류 $\rightarrow$ 정해진 규칙에 따라 번역 
    * 사람과 달리 컴퓨터는 일반화 능력이 매우 떨어지므로 어렵다.
    * 잘 만들어진 규칙에서는 SMT보다 자연스러운 표현 가능
    * 규칙을 일일이 사람이 만드므로 번역기 제작에 많은 자원, 시간 소모
    * 번역 언어쌍 확장시 매번 새로운 규칙을 찾고 적용해야 하므로 매우 불리
   

* 통계 기반 기계번역(SMT)
    * 신경망 기계번역 이전에 지배적인 번역 방식
    * 대량의 양방향 코퍼스에서 통계량을 얻어서 번역 시스템 구성
    * 구글 번역 시스템에 도입
    * 여러 모듈로 구성하여 매우 복잡
    * 언어쌍 확장시 대부분 알고리즘이나 시스템 유지되므로 기존의 RBMT보다 유리

* 딥러닝 이전의 신경망 기계번역
    * 1980년대 NMT 시도 존재
    * 인코더 $\rightarrow$ 디코더 형태의 구조
    * 지금과 같은 성능 불가하여 외면
    * 오토인코더 기반의 신경망 기계번역

* 딥러닝 이후의 신경망 기계번역
    * 현재의 딥러닝 방식(NMT)가 기존의 SMT 크게 앞지른다.
    * 딥러닝 기반 번역기술로 거의 모든 상용 번역기 대체
* 신경망 기계번역의 장점과 잘 동작하는 이유
    * end-to-end 모델
        * SMT는 번역 시스템이 여러가지 모듈로 구성되어 시스템 복잡도가 높아져 훈련하기 어렵다. NMT는 단 하나의 모델로 번역을 해결하여 성능 극대화|
    * 더 나은 언어 모델
        * 신경망 언어 모델(NNLM, Neural Network Language Model) 기반 구조이므로 SMT 기반 n-gram 방식의 언어 모델보다 더 강하다.
        * 희소성 문제가 해결되고 자연스러운 번역 결과 문장의 생성에서 강점을 가진다.
    * 훌륭한 문장 임베딩
        * 신경망으로 문장을 차원 축소하여 임베딩 벡터로 제작하는 능력 우수
        * 단어 단위보다 문장 단위에서 문제가 심각한 노이즈나 희소성 문제 잘 대처 가능
        
<img src = "./asset/chap10/lecture10.png" alt="Example" width="500" height="300"/>

## 10.2 seq2seq

### 10.2.1 구조 소개
* seq2seq 모델 구조를 활용하여 MLE 수행
* 주어진 데이터를 가장 잘 설명하는 파라미터 $\theta$ 찾기

$$\hat{\theta} = \underset{\theta}{argmax}\ P(Y|X;\theta)
\text{ where }=\{x_1, x_2, \cdots, x_n\}, Y=\{y_1, y_2, \cdots, y_m\}$$

$\leftrightarrow P(Y|X;\theta)$ 최대로 하는 모델 파라미터 찾는 작업

* 사후 확률 최대로하는 $Y$ 찾기

$$\hat{Y} = \underset{Y \in \mathcal{Y}}{argmax}\ P(Y|X;\theta)$$

* seq2seq는 크게 3개 서브모듈인 인코더, 디코더, 생성자(generator) 구성
* 인코더
    * 주어진 소스 문장인 여러 벡터를 입력으로 받아 문장을 함축하는 문장 임베딩 벡터로 제작  
    * $\leftrightarrow P(z|X)$ 모델링
    * RNN 분류 모델과 거의 동일하다.
    * $P(z|X)$를 모델링 $\rightarrow$ 주어진 문장을 매니폴드 따라 차원축소 $\rightarrow$ 해당 도메인 잠재 공간의 하나의 점에 투영 작업
    * 기존 텍스트 분류는 모든 정보(특징 feature)가 필요하지 않는다.
    * 반면, 기계번역을 위한 문장 임베딩 벡터를 생성하기 위해 최대한 많은 정보 필요

* 인코더 수식
 $$h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src}) \\
 H^{src} = [h_1^{src};h_2^{src};\cdots;h_n^{src}]$$
 
    * [;] : 이어 붙이는 작업(concatenate)
    * time-step 별로 RNN 통과시켰다.

* 실제 구현
    * 전체 time-step 병렬로 한번에 처리
$$H^{src} = RNN_{enc}(emb_{src}(X),h_{0}^{src})$$

* 디코더
    * 신경망 언어 모델(NNLM) 연장선, 조건부 신경망 언어 모델(CNNLM)
    * seq2seq 모델의 수식을 time-step에 관해 표현
    $$P_{\theta}(Y|X) = \prod_{i=1}^m P_{\theta}(y_t|X, y_{<t}) \\
    \log P_{\theta}(Y|X) = \sum_{i=1}^m \log P_{\theta}(y_t|X, y_{<t})$$
    * RNNLM 조건부 확률 변수에 $X$ 추가
    * 인코더 결과인 문장 임베딩 벡터와 이전 time-step까지 번역하여 생성한 단어 기반하여 현재 time-step 단어 생성
    
    $$h_t^{tgt} = RNN_{dec} (emb_{tgt}(y_{t-1}), h_{t-1}^{tgt}) \\
    \text{where } h_0^{tgt} = h_n^{src}\text{ and }y_0=\text{BOS}$$
    * 디코더 수식으로, 디코더 또한 신경망 언어 모델
    * 디코더 입력 초깃값으로 $y_0$에 BOS 토큰을 입력으로 함.

* 생성자(generator)
    * 디코더에서 각 time-step 별로 결과 벡터 h_t^{tgt} 받아 softmax 계산
    * 각 타깃 언어의 단어별 확률값을 반환하는 단순 작업 수행 모듈
    * 생성자의 결괏값 = 각 단어 나타난 확률, 이산 확률 분포가 된다.
    * 주의할 점
        * 문장 길이 $|Y| = m$일 때, 맨 마지막 반환되는 단어 $y_m$은 EOS 토큰이 된다.
        * 마지막 $y_m$ = EOS, 디코더 계산의 종료이므로 디코더 입력으로 들어가지 않는다.
        
        $$ \hat{y_t} = softmax(linear_{hs\rightarrow |V_{tgt}|}(h_t^{tgt}))\text{ and } \hat{y_m}=EOS \\
        \text{ where }hs \text{ is hidden size of RNN, and }|V_{tgt}|\text{ is size of output vocabulary}$$ 

### 10.2.2 seq2seq 활용 분야
* seq2seq 모델 : 기계번역 문제뿐만 아니라 다양한 분야 적용 가능
    * 시계열 데이터 또는 시퀀스 데이터 입력 / 출력
* seq2seq 응용사례

| 활용 분야 | 입력과 출력 |
|:--------|:--------|
|기계 번역 | 특정 언어 문장 $\rightarrow$ 다른 언어 문장|
|챗봇 | 사용자 문장 $\rightarrow$ 대답|
|요약 | 긴 문장 $\rightarrow$ 같은 언어 요약된 문장|
|기타 자연어 처리 | 사용자 문장 $\rightarrow$ 프로그래밍 코드|
|음성 인식 | 사용자 음성 $\rightarrow$ 해당 언어 문자열(문장)|
|독순술 | 입술 움직임의 동영상 $\rightarrow$ 해당 언어 문장|
|이미지 캡셔닝|  변형된 seq2seq 사용한 이미지 $\rightarrow$ 그림 설명 문장|

### 10.2.3 한계점
* seq2seq는 오토인코더의 일종, 특히 시계열 / 시퀀스 데이터에 강점 있는 모델
* 한계점
    * 장기 기억력
        * 신경망 모델은 차원 축소를 통해 데이터 압축하는 데 탁월한 성능을 가진다.
        * seq2seq 통해도 정보 압축은 한계가 있다.
        * 한계로 인해 문장이나 time-step이 길어질 수록 압축 성능이 떨어진다.
        * LSTM이나 GRU로 RNN보다 높은 성능을 낼 수 있으나 한계가 있다.?
        
    * 구조 정보의 부재
        * 현재 딥러닝 자연어 처리는 문장을 이해할 때 단순히 시퀀스 데이터로 다루는 경향이 있다.
        * 신경망이 알아서 구조 파악 기대
        * 다음 단계에는 구조 정보도 필요하다.        
    * 챗봇 또는 QA봇
        * 당연한 이야기
        * 대화의 흐름에서 대답은 질문보다 새로운 지식이나 문맥정보가 추가된다.
        * 기존 seq2seq 문제(번역이나 요약) 문제는 새롭게 추가된 문제가 없어 잘 해결한다.
        * 대화는 좀 더 발전된 구조 필요

### 10.2.4 파이토치 예제 코드
* 기계번역 seq2seq 파이토치로 구현한다.

* 인코더 클래스
    * 인코더 RNN : 텍스트 분류기 코드와 매우 유사
    * 양방향(Bi-directional) LSTM 사용 $\rightarrow$ 선언시 bidirectional = TRUE
    * 양방향 LSTM 은닉 상태
        * 기존의 단방향(uni-directional) LSTM 은닉 상태보다 2배 증가 $\rightarrow$ hidden_size/2로 RNN hidden_size 이용

In [0]:
import torch
import torch.nn as nn
#인코더 클래스
class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers=4, dropout_p=.2):
        super(Encoder, self).__init__()
        
        # 'batch_first' parameter 값에 유의하라.
        # 또한, hidden_size는 기존 것의 절반으로 양방향이기 때문이다.
        self.rnn = nn.LSTM(word_vec_dim,
                          int(hidden_size / 2),
                          num_layers=n_layers,
                          dropout=dropout_p,
                          bidirectional=True,
                          batch_first=True
                          )
        
    def forward(self, emb):
        # |emb| = (batch_size, length, word_vec_dim)
        
        if isinstance(emb, tuple):
            x, lengths = emb
            x = pack(x, lengths.tolist(), batch_first = True)
            
        else:
            x = emb
        
        y, h = self.rnn(x)
        # |y| = (batch_size, length, hidden_size)
        # |h[0]| = (num_layers * 2, batch_size, hidden_size / 2)
        
        
        if isinstance(emb, tuple):
            y, _ = unpack(y, batch_first = True)
            
        
        return y, h

* pack_padded_sequence 함수
    * 기존의 샘플별 미니배치를 time-step별로 표현한다.
    * PackedSequence로 표현된 time-step별 미니배치는 각 time-step별 샘플의 숫자를 추가적인 정보로 가진다.
    * 미니배치 내 가장 긴 길이의 문장부터 차례로 정렬되어야 한다.

In [0]:
#pack_padded_sequence
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)

In [0]:
b

tensor([[1, 2, 3],
        [3, 4, 0]])

In [0]:
import torch
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=None, unsorted_indices=None)

* 디코더 클래스

* 생성자 클래스

In [0]:
class Generator(nn.Module):

  def __init__(self, hidden_size, output_size):
    super(Generator, self).__init__()

    self.output = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=-1)

  def forward(self, x):

    y = self.softmax(self.output(x))

    return y

* 전체 seq2seq 클래스
생략

* 손실 함수  
  * seq2seq : 기본적으로 각 time-step별 가장 확률이 높은 단어를 선택하는 분류 문제 $\rightarrow$ 교차 엔트로피를 손실 함수로 사용
  * 조건부 언어 모델 $\rightarrow$ 퍼플렉시티를 통해 번역 모델의 성능 표현 가능
  * 교차 엔트로피와 매우 깊은 연관 가짐

* 파이토치에서 손실 함수 준비
  * 실제 구현시 softmax + 교차 엔트로피  $\rightarrow$ logsoftmax + 로그 가능도(NLL)


    loss_weight = torch.ones(output_size)
    loss_weight[data_loader.PAD] = 0

    crit = nn.NLLLoss(weight=loss_weight,
                      reduction='sum',)

* softmax 함수 대신 logsoftmax 함수로 로그 확률을 구한다.
* 나머지 수식 작업 수행

In [0]:
def _get_loss(self, y_hat, y, crit=None):
  crit = self.crit if crit is None else crit
  loss = crit(y_hat.contiguous().view(-1, y_hat.size(-1)),
              y.contiguous().view(-1)
              )
  return loss

## 10.3 어텐션

### 10.3.1 어텐션 이해하기


* 어텐션은 쿼리와 비슷한 값을 가진 키를 찾아 그 값을 얻는 과정
  * json 혹은 프로그래밍 사용하는 key-value 방식과 비교

### 10.3.2 key-value 함수

* 키-밸류 또는 파이썬 딕셔너리 자료형

In [0]:
dic = {'computer':9, 'dog':2,'cat':3}

* key, value 값 입력 후 key를 통해 value 값 접근 가능
* 쿼리가 주어질 때 key 값에 따라 value 값에 접근 가능하다.
* 함수로 나타내면 다음과 같다.

In [0]:
def key_value_func(query):
  weights = []

  for key in dic.keys():
    weights += [is_same(key, query)]

  weight_sum = sum(weights)
  for i, w in enumerate(weights):
    weights[i] = weights[i] / weight_sum
  answer = 0

  for weight, value in zip(weights, dic.values()):
    answer += weight * value

  return answer


def is_same(key, query):
  if key == query:
    return 1. 
  else:
    return .0

* 코드 내부
  * 순차적으로 dic 변수 내부의 key 값들과 query 값을 비교
  * key가 같은 경우 weights 변수에 1.0을 더함
  * key가 다른 경우 weights 변수에 .0을 더함
  * weights를 weights_sum으로 나누어 그 합이 1이 되도록 만든다.
  * 다시 dic 내부의 value값들과 weights값에 대해 서로 곱한 뒤에 더한다.
  * weight가 1.0인 경우에만 value 값을 answer에 더한다.

### 10.3.3 연속적인 key-value 함수
* is_same 함수 대신 다른 함수 쓰기
* key와 query 사이 유사도를 반환하는 how_similar 가상함수 가정

    query = 'puppy'
    how_similar('computer', query)
    # 0.1
    how_similar('dog', query)
    # 0.9
    how_similar('cat', query)
    # 0.7

* key_value 함수 : 이 경우 puppy 단어 테스트 결과는 다음과 같이 실행이 된다.

      query = 'puppy'
      key_value_func(query)
      2.823


$$2.823 = {0.1 \over (0.9 + 0.7 + 0.1)} \times 9 + {0.9 \over (0.9+0.7+0.1)} \times 2 + {0.7 \over (0.9+0.7+0.1)} \times 3$$

* 2.823 값의 의미
  * 강아지와 고양이 컴퓨터 유사도 비율에 따른 dic 값의 비율을 지닌다.
  * is_same 함수 사용시 두 값이 같은지 if 문을 통해 검사 후 값을 할당했다.
    * 0과 1의 불연속적인 값 출력 $\rightarrow$ how_similar 함수를 통해 0에서 1사이의 연속적인 값을 weights 할당 $\rightarrow$ key_value_func 함수 수행
  * key_value_func 딥러닝 사용 가능

### 10.3.4 연속적인 key-value 함수

* dic의 value : 100차원 벡터 구성
* 쿼리의 key 값 모두 벡터 다룬다.
* 단어 임베딩 벡터이다.
* how_similar 함수 : 이 벡터들 간의 코사인 유사도 반환한다.
* dic의 key 값과 value 값이 서로 같다.

* word2vec 함수 : 단어를 입력으로 받아 그 단어에 해당하는 미리 정해진 단어 임베딩 벡터를 반환 가정
  * how_similar 함수 : 두 벡터 간의 코사인 유사도 값 반환

In [0]:
def key_value_func(query):
  weights = []

  for key in dic.keys():
    weights += [how_similar(key, query)]  # 코사인 유사도 반환

  weights = softmax(weights) 
  answer = 0


  for weight, value in zip(weights, dic.values()):
    answer += weight * value

  return answer

* key_value_func : 값을 받아 weight에 저장 $\rightarrow$ 모든 weights 값이 채워지면 softmax 함수 취하기 
  * softmax : weights 합의 크기를 1로 고정시키는 정규화 역할
  * 유사도 총합에서 차지하는 비율만큼 weight의 값이 채워진다.

    from gensim.models.word2vec import Word2Vec
    100
    import gensim.models.word2vec as word2vec
    [0.1, 0.3, -0.7, 0.0 ...]
    len(word2vec('computer'))
    [0.15, 0.2, -0.3, 0.0, ...]

    dic = {word2vec('computer'): word2vec('computer),
            ...}

    query = 'puppy'
    answer = key_Value_func(word2vec(query))

* answer 값에 벡터 값이 들어 간다.
  * puppy 벡터, dog, computer, cat 벡터들의 코사인 유사도에 따라 값이 정해진다.
  * 이 함수는 query와 비슷한 key값을 찾아 비슷한 정도에 따라 weight 정한다.
  * 각 key의 value 값을 weight 값만큼 가져와서 모두 더한다.  
$\therefore$어텐션의 원리

### 10.3.5 기계번역에서의 어텐션
* 번역 과정에서 각 time-step별 출력을 키와 밸류로 삼는다.
* 현재 time-step의 디코더 출력은 쿼리로 삼아 어텐션 계산  

* RNN 기반 seq2seq에서 어텐션이 활용될 때 각 입력값

|항목 |구성 |
|--|--|
|query |현재 time-step의 디코더 출력 |
|keys |각 time-step별 인코더 출력 |
|values |각 time-step별 인코더 출력 |

    context_vector = attention(query = decoder_output,
                           keys = encoder_outputs,
                           values = encoder_outputs)

* 어텐션 추가한 seq2seq 수식

$$ w = softmax(h_t^{tgtT} W \cdot H^{src}) \\
c = H^{src} \cdot \text{ and $c$ is a context vector} \\
\tilde{h_t^{tgt}} = \tanh(linear_{2\times hs $rightarrow hs}([h_t^{tgt};c])) \\
\hat{y_t} = softmax(linear_{hs \rightarrow |V_tgt|}(\tilde{h_t^{tgt}}))
$$

* 원하는 정보를 어텐션을 통해 인코더에서 획득 $\rightarrow$ 해당 정보를 디코더의 출력과 이어붙여 tanh 취한다.
* softmax 계산으로 다음 time-step의 입력이 되는 $\hat{y_t}$ 구한다.

* 선형 변환
  * 각 입력 파라미터 추상적으로 예상

* 기계번역에서 인코더와 디코더 출력값 역할과 의미

|항목 |의미|
|--|--|
|decoder_output |현재 time-step까지 번역된 타깃 언어의 단어 또는 문장, 의미 |
|encoder_output |각 time-step 에서의 소스 언어의 단어 또는 문장, 의미 |

* 신경망 내부의 각 차원 : 숨겨진 특징값(latent feature)이므로 정의 불가
  * 소스 언어와 대상 언어가 애초에 다르다.
  * 단순히 벡터 내적보다 연결고리 추가(소스, 대상언어)
  * 두 언어가 각각 임베딩된 잠재 공간이 선형 관계 있다고 추상적으로 가정
  * 내적 연산을 수행하기 전에 선형 변환을 한다.
  * 이 선형 변환을 위한 $W$값은 신경망 가중치 파라미터, 피드포워드 및 역전파로 학습

* 어텐션의 필요성
  * seq2seq : 인코더와 디코더 두 개의 RNN 구성
  * 문장 임베딩 벡터 해당하는 인코더 결과 벡터 정보를 디코더 은닉 상태(LSTM은 cell state)가 추가로 전달
  * 디코더 : 인코더에서 넘겨받은 은닉 상태로부터 문장 생성
  * 은닉 상태만으로는 문장의 모든 정보를 완벽하게 전달이 어렵다.
    * 문장이 길어질수록 문제가 크다.
  * 디코더의 time-step마다 현재 디코더의 은닉 상태에 따라 필요한 인코더 정보(인코더 마지막 레이어 은닉 상태) 접근하여 끌어다 쓴다.

* 선형 변환자체가 어텐션이다.
  * 선형 변환 과정으로 디코더 현재 상태에 따라 필요한 쿼리 제작
  * 인코더 key 값들과 비교하여 가중합을 한다.
  * 어텐션을 통해 디코더는 인코더에 쿼리를 날린다.
    * 쿼리 잘 날려야 좋은 정답을 얻는다.


* 신경망은 쿼리를 만들어내는 훈련을 하는 것이다.
  * 현재 디코더의 상태에 따라 필요한 정보가 무엇인지를 스스로 판단
  * 선형 변환을 통해 쿼리를 제작할 것이다.
  * 선형 변환을 위한 가중치 파라미터 자체도 한계가 있다.
  * 디코더 상태 자체가 선형 변환이 되어 쿼리가 좋은 형태가 되도록 RNN이 동작할 것이다.

* 어텐션 적용 결과
  * 어텐션 없는 seq2seq 성능이 떨어진다.
  * 문장이 길어질수록 성능이 더욱 하락한다.
  * 어텐션 사용시 문장이 길어져도 성능이 크게 하락하지 않는다.

### 10.3.6 파이토치 예제코드

* torch.bmm 함수 : 배치 행렬 곱(BMM) 수행하는 함수
  * 2개 이상의 차원을 지닌 텐서가 주어졌을 때 뒤의 2개 차원에 대해 행렬 곱을 수행
  * 앞의 다른 차원은 미니배치로 취급한다.
  * 앞의 차원들은 크기가 같아야 하고, 뒤의 2개 차원은 행렬 곱을 수행하기 위한 적절한 크기 필요

* 파이토치 배치 행렬 곱 연산

$$z = torch.bmm(x,y) \\
(batch_size, m, k) \times (batch_size, h. m) = (batch_size, n, m) \\
\Leftrightarrow x \times y = z$$

    import torch
    z = torch.bmm(x,y)

* 어텐션 클래스
  * 선형 변환을 위한 가중치 파라미터를 편향(bias)없는 선형 계층으로 대체

In [0]:
class Attention(nn.Module):

  def __init__(self, hidden_size):
    super(Attention, self).__init__()

    self.linear = nn.Linear(hidden_size, hidden_size, bias=False)
    self.softmax = nn.Softmax(dim=-1)


  def forward(self, h_src, h_t_tgt, mask=None):
    query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)

    weight = torch.bmm(h_src, query).squeeze(-1)

    if mask is not None:

      weight.masked_fill_(mask, -float('inf'))
    weight = self.softmax(weight)

    context_vector = torch.bmm(weight.unsqueeze(1), h_src)

    return context_vector
